In [145]:
from itertools import combinations
import numpy as np
import random
from icecream import ic
from typing import List, NamedTuple

## Simple Test Problem

In [146]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('lab2/test_problem.npy')

## Common tests

In [147]:
problem = np.load('lab2/problem_r1_200.npy')

In [148]:
# Negative values?
np.any(problem < 0)

False

In [149]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

True

In [150]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [151]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False

In [152]:
class Individual(NamedTuple):
    tour: List[int]
    fitness: float | None = None

def fitness(tour, dist: np.ndarray) -> float:
    """compute total tour fitness (closed loop)"""
    t = np.array(tour)
    return float(np.sum(dist[t, np.roll(t, -1)]))

def make_individual(solution: List[int], distance_matrix: np.ndarray) -> Individual:
    return Individual(tour=solution, fitness=fitness(solution, distance_matrix))

def generate_random_solution(n_cities: int) -> List[int]:
    """generate a random permutation of cities using numpy"""
    return np.random.permutation(n_cities).tolist()

def generate_population(pop_size: int, n_cities: int, dist: np.ndarray) -> List[Individual]:
    """generate initial random population sorted by fitness"""
    population = [make_individual(generate_random_solution(n_cities), dist) for _ in range(pop_size)]
    population.sort(key=lambda x: x.fitness)
    return population

# --- PARENT SELECTION ---
def tournament_selection(population: List[Individual], k: int = 3) -> Individual:
    """ 
    k controls selective pressure:
    - high k -> the real fittest more likely to win -> more selective pressure (converges faster)
    - low k -> more chances to win for anyone -> less selective pressure, more random (... slower)
    the idea is to increase k for smaller instances of the tsp problem and decrease for bigger ones
    """
    competitors = random.sample(population, k)
    winner = min(competitors, key=lambda x: x.fitness)
    return winner

# alternative
def rank_selection(population: List[Individual], p: float = 1.0) -> Individual:
    """
    rank-based selection tweaking selective pressure with p (requires initial sorting)
    - p = 0  -> uniform random (each individual has the same probability to be selected)
    - p = 1  -> linear ranking 
    - p > 1  -> exponential ranking (more selective pressure, more chances for the fittest)
    """
    N = len(population)
    ranks = np.arange(N, 0, -1) # [N, N-1, ..., 1]
    weights = ranks ** p # (r_i ^ p)
    probs = weights / weights.sum() # normalize weights to probs
    idx = np.random.choice(N, p=probs)
    return population[idx] # return parent
    
# --- debug ---
# n_cities = len(CITIES)
# pop = generate_population(pop_size=10, n_cities=n_cities, distance_matrix=test_problem)
# ic(pop)


In [153]:
# --- MUTATION FUNCTIONS ---
def mutate_swap(solution: List[int]) -> List[int]:
    """swap two random cities"""
    sol = solution.copy()
    i, j = np.random.choice(len(sol), 2, replace=False)
    sol[i], sol[j] = sol[j], sol[i]
    return sol

def mutate_insertion(solution: List[int]) -> List[int]:
    """remove a random city and reinsert it in another random position"""
    sol = solution.copy()
    i, j = np.random.choice(len(sol), 2, replace=False)
    city = sol.pop(i)
    sol.insert(j, city)
    return sol

def mutate_inversion(solution: List[int]) -> List[int]:
    """reverse a random segment (should be more efficient for instances with symmetrical matrix)"""
    sol = solution.copy()
    i, j = sorted(np.random.choice(len(sol), 2, replace=False))
    sol[i:j+1] = reversed(sol[i:j+1])
    return sol

# test
def random_mutation(sol: List[int]) -> List[int]:
    ops = [mutate_swap, mutate_insertion, mutate_inversion]
    op = random.choice(ops)
    return op(sol)

# --- RECOMBINATION/CROSSOVER FUNCTIONS ---
def order_crossover(p1, p2):
    """copy a random slice from p1 then fill the remaining positions from p2 in order"""
    size = len(p1)
    a, b = sorted(np.random.choice(size, 2, replace=False))
    child = [None] * size
    child[a:b] = p1[a:b] # [a,b)
    fill = [c for c in p2 if c not in child]
    idx = [i for i in range(size) if child[i] is None]
    for i, c in zip(idx, fill):
        child[i] = c
    return child

def inver_over_v1(p1: List[int], p2: List[int]) -> List[int]:
    """ steps:
    1. select a random starting city from p1
    2. find its successor in the guiding parent p2 
    3. if the edge already exists in p1 -> stop
    4. otherwise, invert the segment between the two cities in p1
    (should work very well with symmetric tsp)
    """
    tour = p1.copy()
    n = len(tour)
    city = random.choice(tour) # step 1: random starting city
    idx2 = p2.index(city) 
    next_city = p2[(idx2 + 1) % n] # step 2: find successor in guiding parent
    i, j = tour.index(city), tour.index(next_city)
    if (i + 1) % n == j or (j + 1) % n == i:
        return tour  # step 3: if they are already adjacent -> stop (nothing to do)
    a, b = sorted([i, j])
    tour[a+1:b+1] = list(reversed(tour[a+1:b+1]))  # step 5: otherwise invert the segment between them
    return tour

# just an alternative, inver-over used as single operator
def inver_over_v2(p1: List[int], p2: List[int], prob: float = 0.02) -> List[int]:
    tour = p1.copy()
    n = len(tour)
    city = np.random.choice(tour)
    while True: 
        if np.random.random() < prob: # select from p2 (like crossover)
            idx2 = p2.index(city)
            next_city = p2[(idx2 + 1) % n]
        else: # select from p1 (more like mutation)
            next_city = np.random.choice([c for c in tour if c != city])
        i, j = tour.index(city), tour.index(next_city)
        if (i + 1) % n == j or (j + 1) % n == i:
            break # stop condition
        a, b = sorted([i, j])
        tour[a+1:b+1] = list(reversed(tour[a+1:b+1]))
        city = next_city
    return tour

def opt2_localsearch(tour: list[int], dist: np.ndarray) -> list[int]:
    best = np.array(tour, dtype=int)
    n = len(best)
    best_cost = np.sum(dist[best, best[np.roll(np.arange(n), -1)]])
    improved = True

    while improved:
        improved = False
        for i in range(n - 2):
            a, b = best[i], best[(i + 1) % n] # first edge (a.b)
            for j in range(i + 2, n): # second edge to cut (c,d)
                if j == n - 1 and i == 0:
                    continue
                c, d = best[j], best[(j + 1) % n]
                # remove (a,b) and (c,d), add (a,c) and (b,d)
                delta = (dist[a, c] + dist[b, d]) - (dist[a, b] + dist[c, d])
                if delta < -1e-12: # delta < 0 => new tour is shorter, improvement -> stop
                    best[i+1:j+1] = best[i+1:j+1][::-1]
                    best_cost += delta
                    improved = True
                    break
            if improved:
                break
    return best.tolist()


In [154]:
def steady_state(population, distance_matrix, 
                 POP_SIZE: int, generations: int, k: int, rate: float):
    """steady-state approach (replaces the worst individual only if offspring is better)"""

    population.sort(key=lambda x: x.fitness)
    for step in range(generations):
        # --- generate offspring ---
        if np.random.random() < rate:
            p = tournament_selection(population, k)
            child = mutate_swap(p.tour)
            # child = mutate_insertion(p.tour)
            # child = mutate_inversion(p.tour)
            # child = random_mutation(p.tour)
        else:
            p1, p2 = tournament_selection(population, k), tournament_selection(population, k)
            # child = inver_over_v1(p1.tour, p2.tour)
            child = order_crossover(p1.tour, p2.tour)

        child = make_individual(child, distance_matrix)
        # --- replace only if better than worst ---
        if child.fitness < population[-1].fitness:
            population[-1] = child
            population.sort(key=lambda x: x.fitness)  # keep sorted

        if step % 1000 == 0:
            print(f"Step {step:5d} | best = {population[0].fitness:.2f}")
    return population[0] # best individual

In [155]:
def generational(population, distance_matrix, 
                 POP_SIZE: int, rate: float, elitism: int, generations: int, k: int = 3):
    """survival selection (generational approach + some elitism)"""

    prev_best = population[0].fitness
    stagnation = 0 
    # the population was previously sorted
    for gen in range(generations):
        elites = population[:elitism] # some elitism (preserve best)
        new_population = elites.copy()
        while len(new_population) < POP_SIZE:
            if np.random.random() < rate:
                p = tournament_selection(population, k)
                child = mutate_swap(p.tour)
                # child = mutate_insertion(p.tour)
                # child = mutate_inversion(p.tour)
                # child = random_mutation(p.tour)
            else:
                p1, p2 = tournament_selection(population, k), tournament_selection(population, k)
                # child = inver_over_v1(p1.tour, p2.tour)
                child = order_crossover(p1.tour, p2.tour)
            new_population.append(make_individual(child, distance_matrix))
            # the population is entirely replaced (except for elitism)
        new_population.sort(key=lambda x: x.fitness)
        population = new_population[:POP_SIZE] 
        
        if gen % 100 == 0:
            print(f"Gen {gen:5d} | best = {population[0].fitness:.2f}")
        
        best = population[0].fitness
        if abs(prev_best - best) < 1e-6: # no significant difference
            stagnation += 1
        else:
            stagnation = 0
        prev_best = best
        
        if stagnation > 500:  
            print(f"Converged early at generation {gen}")
            break

    return min(population, key=lambda x: x.fitness)

In [156]:
def hybrid(population, distance_matrix, 
           POP_SIZE: int, generations: int, k: int, elitism: int, rate: float, OFFSPRING_SIZE: int):
    """first generate all the OFFSPRINGS, then add it to the population and they compete for survival"""
    prev_best = population[0].fitness
    stagnation = 0
    # the population was previously sorted
    for gen in range(generations):
        elites = population[:elitism] # some elitism
        # --- generate λ offspring ---
        λ = []
        while len(λ) < OFFSPRING_SIZE:
            if np.random.random() < rate:
                p = tournament_selection(population, k)
                child = mutate_swap(p.tour)
                # child = mutate_insertion(p.tour)
                # child = mutate_inversion(p.tour)
                # child = random_mutation(p.tour)
            else:
                p1, p2 = tournament_selection(population, k), tournament_selection(population, k)
                # child = inver_over_v1(p1.tour, p2.tour)
                child = order_crossover(p1.tour, p2.tour)    
            λ.append(make_individual(child, distance_matrix))

        # --- merge λ and elites into the population ---
        population.extend(λ)
        population.extend([e for e in elites if e not in population])
        population.sort(key=lambda x: x.fitness)
        population = population[:POP_SIZE] # sort and keep best individuals
        
        if gen % 100 == 0:
            print(f"Gen {gen:5d} | best = {population[0].fitness:.2f}")  

        best = population[0].fitness
        if abs(prev_best - best) < 1e-6: # no significant difference
            stagnation += 1
        else:
            stagnation = 0
        prev_best = best
        
        if stagnation > 500:  
            print(f"Converged early at generation {gen}")
            break

    return min(population, key=lambda x: x.fitness)

In [157]:
def hybrid_v2(population, distance_matrix, 
           POP_SIZE: int, generations: int, k: int, elitism: int, mutation_rate: float, OFFSPRING_SIZE: int,
           local_search_rate: float = 0.1):
    """the idea here is to improve the first hybrid model with some additional local search:
        - NOT after every mutation/crossover (really too slow for large instances, even if only for 10% offsprings)
        - use it on the best few individuals every N generations (e.g. every 100) and never in early generations
        - trigger it at the end, after stagnation (i.e. no improvement for many gens)
        never in early generations otherwise the model converges to a local optima too fast
        this keeps diversity early and refines tours later when population stabilizes
    """
    prev_best = population[0].fitness
    stagnation = 0
    for gen in range(generations):
        elites = population[:elitism] 
        offspring = []
        # --- generate λ offspring ---
        while len(offspring) < OFFSPRING_SIZE:
            if np.random.random() < mutation_rate:
                parent = tournament_selection(population, k)
                child_tour = mutate_swap(parent.tour)
                # child_tour = mutate_inversion(parent.tour)
            else:
                p1 = tournament_selection(population, k)
                p2 = tournament_selection(population, k)
                # child_tour = inver_over_v1(p1.tour, p2.tour)
                child_tour = order_crossover(p1.tour, p2.tour)
            
            # additional local search (! this is really slow)
            # if np.random.random() < local_search_rate:
            #   child_tour = opt2_localsearch(child_tour, distance_matrix)

            offspring.append(make_individual(child_tour, distance_matrix))

        # --- combine and sort population ---
        population.extend(offspring)
        population.extend(elites)
        population.sort(key=lambda x: x.fitness)
        population = population[:POP_SIZE]

        if gen % 100 == 0:
            """
            # --- late-stage 2-opt refinement only on best individuals ---
            if gen % 500 == 0 and gen > generations * 0.3:  # after 30% of run (never in early generations)
                print(f"Refining with 2-opt at gen {gen}")
                top_k = min(3, len(population))
                for i in range(top_k):
                    improved_tour = opt2_localsearch(population[i].tour, distance_matrix)
                    population[i] = make_individual(improved_tour, distance_matrix)
                # re-sort after refinement
                population.sort(key=lambda x: x.fitness)
            """
            print(f"Gen {gen:5d} | best = {population[0].fitness:.2f} | mut={mutation_rate:.2f} | k={k}")   
            
        best_fit = population[0].fitness
        if abs(prev_best - best_fit) < 1e-6:
            stagnation += 1
        else:
            stagnation = 0
        prev_best = best_fit

        # dynamic parameters
        if stagnation > 100:
            mutation_rate = max(mutation_rate * 0.95, 0.2) # slightly increase crossover
            if k > 2:
                k -= 1  # reduce selection pressure to increase diversity
        if gen % 1000 == 0:
            mutation_rate = min(mutation_rate * 1.1, 0.9) # slightly increase mutation
            k = min(k + 1, 5) # increase selection pressure 

        if stagnation > 500:  
            print(f"Refining with 2-opt after stagnation")
            top_k = min(10, len(population))
            for i in range(top_k):
                improved_tour = opt2_localsearch(population[i].tour, distance_matrix)
                population[i] = make_individual(improved_tour, distance_matrix)
            # re-sort after refinement
            population.sort(key=lambda x: x.fitness)
            print(f"Converged early at generation {gen}")
            break

    return population[0]

In [158]:
# just a test
def pure_inver_over(population, distance_matrix, generations: int = 1000, k: int = 3, prob: float = 0.02):
    """pure inver-over as single operator"""
    n = len(population[0].tour)
    for gen in range(generations):
        new_pop = []
        for ind in population:
            p2 = tournament_selection(population, k)
            child_tour = inver_over_v2(ind.tour, p2.tour, prob)
            new_pop.append(make_individual(child_tour, distance_matrix))

        population.extend(new_pop)
        population.sort(key=lambda x: x.fitness)
        population = population[:len(new_pop)]  # to keep pop size fixed

        if gen % 100 == 0:
            print(f"Gen {gen:4d} | best = {population[0].fitness:.2f}")
    return population[0]

In [159]:
def run_tsp(distance_matrix,
            model: str = "hybrid",
            pop_size: int = 100,
            generations: int = 1000,
            k: int = 3,
            elitism: int = 5,
            rate: float = 0.3,
            offspring_size: int = 20):
    """
    run a TSP evolutionary algorithm using the specified model
    model ∈ {'steady', 'generational', 'hybrid', 'inver-over'}
    """
    n_cities = len(distance_matrix)
    population = generate_population(pop_size, n_cities, distance_matrix)
    print(f"Initial best = {population[0].fitness:.2f}")
    
    if model == "steady":
        best = steady_state(population, distance_matrix, pop_size, generations, k, rate)
    elif model == "generational":
        best = generational(population, distance_matrix, pop_size, rate, elitism, generations, k)
    elif model == "hybrid":
        best = hybrid(population, distance_matrix, pop_size, generations, k, elitism, rate, offspring_size)
    elif model == "inver-over":
        best = pure_inver_over(population, distance_matrix, generations, k)
    else:
        raise ValueError("model name not correct'")

    print(f"\nFinal best fitness: {best.fitness:.2f}")
    return best

In [160]:
best = run_tsp(problem,
               model="hybrid",  # 'steady', 'generational', or 'hybrid'
               pop_size=100,
               generations=10_000,
               elitism=5,
               k=3,
               rate=0.8,
               offspring_size=20)

print("Best tour:", best.tour)

Initial best = 9492.00
Gen     0 | best = 9492.00
Gen   100 | best = 7090.54
Gen   200 | best = 6143.93
Gen   300 | best = 5676.35
Gen   400 | best = 5361.89
Gen   500 | best = 5117.78
Gen   600 | best = 4910.88
Gen   700 | best = 4744.98
Gen   800 | best = 4556.25
Gen   900 | best = 4477.82
Gen  1000 | best = 4420.46
Gen  1100 | best = 4344.81
Gen  1200 | best = 4226.60
Gen  1300 | best = 4189.97
Gen  1400 | best = 4116.07
Gen  1500 | best = 3980.86
Gen  1600 | best = 3912.96
Gen  1700 | best = 3831.80
Gen  1800 | best = 3752.34
Gen  1900 | best = 3686.81
Gen  2000 | best = 3609.52
Gen  2100 | best = 3582.53
Gen  2200 | best = 3555.95
Gen  2300 | best = 3510.70
Gen  2400 | best = 3468.26
Gen  2500 | best = 3446.19
Gen  2600 | best = 3392.66
Gen  2700 | best = 3336.16
Gen  2800 | best = 3294.52
Gen  2900 | best = 3260.04
Gen  3000 | best = 3228.56
Gen  3100 | best = 3184.53
Gen  3200 | best = 3158.58
Gen  3300 | best = 3133.83
Gen  3400 | best = 3112.35
Gen  3500 | best = 3071.94
Gen  